# Transfer Learning for VGG16 Model with RGB

This notebook includes code for finetunning a VGG16 model for classification with Keras on the RGB version of the eurosat dataset.

In [ ]:
# Uncomment if running on mac GPU
# %pip install tensorflow-macos tensorflow-metal

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np

## Preprocess images 
Download the RGB version of the eurosat database from tensorflow.datasets. Dataset includes 27,000 examples. 

In [ ]:
# Load EuroSAT dataset
ds, ds_info = tfds.load('eurosat', split='train', with_info=True, as_supervised=True)
total = 27000
train_size = int(0.8*total)
val_size = int(0.1*total)
test_size = total - train_size - val_size

train_dataset = ds.take(train_size)
train_labels = ds_info.take(train_size)
val_dataset = ds.take(val_size)
val_labels = ds.take(val_size)
test_dataset = ds.take(test_size)
test_labels = ds_info.take(test_size)

Visualize an image from the training set

In [ ]:
for image, label in ds.take(1):
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(f"Label: {label.numpy()}")
    plt.axis("off")
    plt.show()

Preprocess the datasets to match the image size expected from VGG16.

1. Resizes the image to (224, 224) pixels, which is the input size expected by VGG16.

2. Applies preprocess_input from tensorflow.keras.applications.vgg16, which: 
    - converts the image to float32
    - Changes the color channel order from RGB to BGR
    - Subtracts the mean pixel values (specific to ImageNet: [103.939, 116.779, 123.68] for B, G, R channels).
3. Applies prefetch autotune, which allows tensorflow to prepare the next batch as the current one is loading for max efficiency. 

In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))  # match VGG16 input size
    image = preprocess_input(tf.cast(image, tf.float32))
    return image, label

train_dataset = train_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


## Fine-tune model 

Load the model (in this case VGG16 trained on imagenet) and extract it's convolutional base. These are the layers that extract the features from the images. 

We then freeze these layers from training and plug in a new classifier layer at the end with a custom number of classes. 

In [ ]:
# Get the convolutional layers from the VGG16 model and configure to needs. 
conv_base = tf.keras.applications.VGG16(weights='imagenet', 
                                        include_top=False, 
                                        input_shape=(224, 224, 3))

The preprocess_input function ensures satellite images are formatted correctly so the model can extract meaningful features. It iterates over all the batches and returns two numpy arrays:   

1) The features, whose shape depends on the output of your conv_base (Samples, Height, Width, Channels).

2) The labels, which are stored in a one-dimensional array or two-dimensional (if one‐hot encoded).

In [ ]:
def get_features_and_labels(dataset, conv_base):
    conv_base.trainable = False
    processed_images = preprocess_input(dataset)
    return processed_images

Now build the classifier layers that go ontop of the frozen convolutional base. 

In [ ]:
def build_dense_model(input_shape= train_features.shape[1:]): #input shape is (height, width, channels) because keras is agnostic to batch size

    inputs = keras.Input(shape=(input_shape))
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256)(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation='sigmoid')
    model = keras.Model(inputs, outputs)
    model.compile(loss='sparse_crossentropy', # for non-binary tasks 
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

Run the model and visualize the loss and accuracy throughout the training loop.

In [ ]:
dense_model = build_dense_model(input_shape=train_dataset.shape[1:])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = dense_model.fit(
    train_dataset, train_labels,
    epochs=20,
    validation_data=(val_dataset, val_labels),
    callbacks=callbacks)

# Plot training & validation loss values
plt.figure(figsize=(12, 4))

# Accuracy subplot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss subplot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Test model 
Evaluate on test dataset 

In [ ]:
dense_model.evaluate(test_dataset, verbose=1)
